# Question 2

In [1]:
# for basic file navigation and driving mount
import os
from google.colab import drive

# some basic libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# importing for basic image operations
from PIL import Image
from PIL import ImageOps

# importing pretrained models
from tensorflow.keras.applications import efficientnet_v2
from tensorflow.keras.datasets import mnist


# for splitting data
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# prerequisites for training models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from sklearn.svm import SVC

# for selecting better metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [2]:
# define TPU strategy for effecient computation
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()

Device: grpc://10.117.98.186:8470
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.117.98.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.117.98.186:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
# load data from cifar 10 dataset from default train test split (around 85:15)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# convert to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
with strategy.scope():

  # define a sequential model
  model = Sequential([
                      tf.keras.layers.InputLayer(input_shape = (28, 28, 1)),
                      tf.keras.layers.BatchNormalization(),
                      tf.keras.layers.Conv2D(filters = 20, kernel_size = (3,3), activation = "relu", padding = "same"),
                      tf.keras.layers.MaxPool2D(strides = 2),
                      tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), activation = "relu", padding = "same"),
                      tf.keras.layers.MaxPool2D(strides = 2),
                      tf.keras.layers.Flatten(),
                      tf.keras.layers.Dense(32, activation = tf.keras.layers.LeakyReLU(alpha=0.01)),
                      tf.keras.layers.Dense(10, activation = "softmax")
                    ])
  model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate = 0.0001), loss= tf.keras.losses.CategoricalCrossentropy(from_logits = False), metrics=[tf.keras.metrics.CategoricalAccuracy(name = "cat_acc")])
  model.fit(x_train, y_train, epochs = 2, batch_size = 32)

Epoch 1/2
1875/1875 [==============================] - 26s 12ms/step - loss: 0.5707 - cat_acc: 0.8329
Epoch 2/2
1875/1875 [==============================] - 23s 12ms/step - loss: 0.1640 - cat_acc: 0.9517


In [5]:
with strategy.scope():

  # define a new model that takes input and gives out CNN features
  model2 = Sequential([tf.keras.layers.InputLayer(input_shape = (28, 28, 1)),
                    tf.keras.layers.BatchNormalization(),
                    tf.keras.layers.Conv2D(filters = 20, kernel_size = (3,3), activation = "relu", padding = "same"),
                    tf.keras.layers.MaxPool2D(strides = 2),
                    tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), activation = "relu", padding = "same"),
                    tf.keras.layers.MaxPool2D(strides = 2),
                    tf.keras.layers.Flatten()
                    ])
  
  # set the model weights and biases the same as from the trained model
  model2.layers[1].set_weights(model2.layers[1].get_weights())
  model2.layers[3].set_weights(model2.layers[3].get_weights())

  # pass training data through CNN layers and get CNN features
  predictions = model2(x_train).numpy()

In [11]:
# define a default Support vector classifier
svc = SVC()

# pass training data through CNN layers and get CNN features
svc.fit(predictions, y_train.argmax(axis = 1))

SVC()

In [14]:
# try on testing data

# pass testing data through CNN layers and get CNN features
with strategy.scope():
  predictions = model2(x_test).numpy()

# get predictions from svc
predictions = svc.predict(predictions)

# evaluate the performance
print(classification_report(predictions, y_test.argmax(axis = 1)))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       992
           1       1.00      0.99      0.99      1143
           2       0.98      0.98      0.98      1028
           3       0.99      0.98      0.98      1019
           4       0.98      0.99      0.99       974
           5       0.98      0.98      0.98       889
           6       0.98      0.99      0.99       948
           7       0.98      0.98      0.98      1029
           8       0.97      0.97      0.97       975
           9       0.97      0.98      0.98      1003

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



# Question 2